In [ ]:
!apt-get update
!apt-get install -y tesseract-ocr libtesseract-dev

!pip install -U torch torchvision

!pip install -U cython
!pip install -U 'git+https://github.com/facebookresearch/fvcore.git'
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

!git clone https://github.com/facebookresearch/detectron2.git
%cd detectron2
!pip install -e .
%cd ..

!pip install -U matplotlib numpy opencv-python-headless pandas tqdm tabulate gradio streamlit huggingface_hub pytesseract

import torch
import torchvision
import detectron2
#import encapsu_view
import gradio as gr
import cv2
import matplotlib.pyplot as plt
import pytesseract

print("PyTorch:", torch.__version__)
print("Detectron2:", detectron2.__version__)

In [ ]:
!pip install git+https://github.com/MaksymTymkovych/alginate-capsules-analysis
